In [1]:
import requests, os, json
import pandas as pd
from bs4 import BeautifulSoup as bs

In [ ]:
main_url = "https://www.4icu.org/reviews/index{}.htm"
to_format = "https://www.4icu.org/reviews/{}"

In [ ]:
def name_ranking(tags):
    uni = tags.find("div", {"class": "panel panel-default text-center"})
    name = uni.find("h1", {"itemprop": "name"}).text
    rank = uni.find_all("tr", {"style": "vertical-align:bottom"})
    a = {}
    for el in rank:
        tds = el.find_all("td")
        assert len(tds) == 2, ValueError
        k,v = tds
        k, v = k.text, v.text
        a.update({k.replace(" ","_").strip(): v})
    return name, a

def identity(tags, url):
    
    my_list = ["Name", "Acronym", "Founded"]
    a = {"url":url}
    if not tags:
        return
    
    for c in tags.find_all("tr"):
        th, td = c.find("th"), c.find("td")
        if not th or not td:
            continue
        elif th and td:
            th, td = th.text, td.text
            if th in my_list:
                a.update({th.lower(): td})
    return a

def location(tags):
    a = {}
    t = tags
    loc = t.find("span", {"itemprop": "addressLocality"})
    region = t.find("span", {"itemprop": "addressRegion"})
    if loc:
        a.update({"city": loc.text.strip()})
    if region:
        a.update({"region": region.text.strip()})
    
    td = t.find("td")
    if td:
        for s in td.find_all("span"):
            s.extract()
        a.update({"state": td.text.strip()})
    return a

def size_profile(tags):
    
    my_list = ["Student Enrollment", "Control Type", "Entity Type"]
    a = {}
    if not tags:
        return
    
    for c in tags.find_all("tr"):
        th, td = c.find("th"), c.find("td")
        if not th or not td:
            continue
        elif th and td:
            th, td = th.text, td.text
            if th in my_list:
                a.update({th.lower().replace(" ", "_").strip(): td.strip()})
    return a

In [ ]:
f = main_url.format(2)
content = bs(requests.get(f).content, "lxml")
pages = [c.find("a").get("href") for c in  content.find("ul", {"class":"pagination pagination-sm"}).find_all("li")]
main = {}
errors = []

for page in pages:
    
    cc = bs(requests.get(to_format.format(page)).content, "lxml")
    table = cc.find("table", {"class":"table table-hover text-left"}).find("tbody")
    infos = [c.find("a").get("href").split("/")[-1] for c in table.find_all("tr")]
    
    my_list = ["University Identity", "University Location", "Size and Profile"]
    
    for inf in infos: 
        try:
            content = bs(requests.get(to_format.format(inf)).content, "lxml")
            selected_panels = {}
            for panel in content.find_all("div", {"class": "panel panel-default"}):
                title = panel.find("div", {"class": "panel-heading"})
                if title:
                    title = title.find("h2").text
                    if title in my_list:
                        selected_panels.update({title: panel})
            name, rank = name_ranking(content)
            id_ = identity(selected_panels.get(my_list[0]), to_format.format(inf))
            loc = location(selected_panels.get(my_list[1]))
            sp = size_profile(selected_panels.get(my_list[2]))


            tot = {**id_, **rank, **loc, **sp}
            main.update({name.strip(): tot})
            print(f"Successfully updated: {name.strip()}")
        except Exception as e:
            errors.append(to_format.format(inf))
            print(f"Error on {to_format.format(inf)}, type: {e}")
            #break

In [ ]:
with open("./universities.json", "w") as f:\
    json.dump(main, f)

In [ ]:
df = pd.DataFrame.from_dict(main, orient="index")

In [ ]:
df["main_name"] = df.index

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.to_csv("./universities.csv", index=False)